In [1]:
import pandas as pd
from sklearn import preprocessing
from soyviz import plot_los_tres
from combat.pycombat import pycombat
import matplotlib.pyplot as plt

In [ ]:
venancio = pd.read_parquet('../expresion/venancio.parquet')
venancio

In [ ]:
annot = pd.read_csv('../anotacion/gemini_annot.csv', index_col=0)
annot = annot[annot['tejido'].notna() & annot['estres'].notna()]
annot

In [ ]:
annot = annot.loc[(annot['tejido'] == 'leaf') | (annot['tejido'] == 'root') | (annot['tejido'] == 'seed') | (annot['tejido'] == 'stem')]
annot

In [ ]:
venancio, annot = venancio.align(annot, join='inner', axis=0)
venancio

In [22]:
plot_los_tres(venancio, annot)
plt.suptitle('Expresión cruda')

## Filtro observaciones y features

Varianza 0

In [ ]:
annot['tejido'].value_counts()

In [ ]:
annot['estres'].value_counts()

In [ ]:
consamples = annot['bioproject'].map(annot['bioproject'].value_counts() > 2)
conestres = annot['estres'].map(annot['estres'].value_counts() > 10)
convarianza = venancio.var() > 0.01

filtrado = venancio.loc[consamples & conestres, convarianza]

filtrado

In [31]:
filtrado, annot = filtrado.align(annot, join='inner', axis=0)
filtrado

In [ ]:
plot_los_tres(filtrado, annot)
plt.suptitle('Expresión cruda')

## Batch correction

In [ ]:
combat = pycombat(filtrado.T, annot['bioproject']).T
combat

In [ ]:
estandar = filtrado.groupby(annot['bioproject']).apply(lambda g: pd.DataFrame(preprocessing.scale(g), index=g.index, columns=g.columns)).droplevel('bioproject')
# estandar = estandar.groupby(condicion['tejido']).apply(lambda g: pd.DataFrame(preprocessing.scale(g), index=g.index, columns=g.columns)).droplevel('tejido')
estandar

In [ ]:
annot = annot.drop(combat[combat.isna().sum(axis=1) > 0].index)
estandar = estandar.drop(combat[combat.isna().sum(axis=1) > 0].index)
combat = combat.drop(combat[combat.isna().sum(axis=1) > 0].index)

annot

In [ ]:
plot_los_tres(estandar, annot)
plt.suptitle('Expresión estandarizada')

In [ ]:
plot_los_tres(combat, annot)
plt.suptitle('Expresión ComBat')

In [22]:
combat.to_parquet('combat.parquet')
estandar.to_parquet('estandar.parquet')
annot.to_parquet('condicion.parquet')